In [ ]:
#@title 檢視 GPU
! nvidia-smi

Thu Mar 17 16:18:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@title 安裝 colorama, ffmpeg

! pip install colorama
! pip install ffmpeg-python

In [ ]:
#@title 安裝 DeepFaceLab
!git clone https://github.com/iperov/DeepFaceLab.git

Cloning into 'DeepFaceLab'...
remote: Enumerating objects: 8010, done.
remote: Counting objects: 100% (259/259), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 8010 (delta 146), reused 216 (delta 131), pack-reused 7751
Receiving objects: 100% (8010/8010), 823.27 MiB | 33.79 MiB/s, done.
Resolving deltas: 100% (5142/5142), done.
Checking out files: 100% (211/211), done.


In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title 防止 Google Colab 斷開連線

import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


In [ ]:
#@title 將素材從 Google Drive 搬回 VM
workspace_file = '/content/drive/MyDrive/aidataset/deepfacelab/deepfacelab_ws2.zip' #@param {type:"string"}
cmd  = f'cp {workspace_file} ./'
! $cmd

In [ ]:
#@title 解壓縮Workspace
zip_file = 'deepfacelab_ws2.zip' #@param {type:"string"}
cmd  = f'unzip  {zip_file}'
! $cmd

## 可以在 https://dfldata.xyz/ 搜尋萬用 XSeg 模型

In [ ]:
#@title 套用或移除 XSeg 遮罩
Mode = "Apply mask" #@param ["Apply mask", "Remove mask"]
Data = "data_src" #@param ["data_src", "data_dst"]

main_path = "/content/DeepFaceLab/main.py "
data_path = "/content/workspace/"+Data+"/aligned "
mode_arg = "apply " if Mode == "Apply mask" else "remove "
cmd = main_path+"xseg "+mode_arg+"--input-dir "+data_path
cmd += "--model-dir /content/workspace/model" if mode_arg == "apply " else ""

!python $cmd

In [ ]:
#@title 訓練 DeepFakes 模型
import os
model_path = '/content/drive/MyDrive/aidataset/deepfacelab/model_20220312' #@param {type:"string"}
if not os.path.exists(model_path):
  os.mkdir(model_path)
%cd /content/DeepFaceLab/
cmd = f'python main.py train --training-data-src-dir /content/workspace/data_src/aligned --training-data-dst-dir /content/workspace/data_dst/aligned  --model-dir {model_path} --model SAEHD'
! $cmd